This program give information on how much one will need to retire.

add inflation
add tax estimate
add statistics - monte carlo analysis
add graphics

answers question - when will a person run out of money
                 - how much will they need to save not so they won't run out of money
                 - how long will they have to delay retirement so they won't run out of money
                 - how much will they will have to reduce their need so they won't run out of money
                
                


In [2]:
import pandas as pd
import numpy as np
import time

In [102]:
###################################################
# METHODS
###################################################

#inputs all of the needed information and return two lists and a df
def var_input():
    
    variables = []
    variable_names = []
    
    #time to retirement
    years_to_retire = int(input('In how many years do you plan to retire? '))
    
    #amount needed at retirement
    how_much_needed_at_retirement = float(input('How much per year will you need at retirement? [Before taxes.]'))
    
    #current retirement savings
    starting_principle = float(input('How much have you saved for retirement right now? [in thousands]'))
        
    #income after retirment
    how_much_ss_at_retirement = float(input('How much Social Security benefits per year will you get at retirement? [in thousands/year] '))
    how_much_other_passive_income = float(input('How much other passive income per year will you get at retirement? [e.g. Pension; in thousands/year] '))
    
    #return on retirement savings
    investment_apr = float(input('What is the annualized percent return on your retirement saving before retiring? '))

    #life expectance after retirement
    yrs_to_death = input('How long will you need retirement income? [press enter if you want 20 yrs.]')
    if yrs_to_death:
        yrs_to_death = float(yrs_to_death)
    else:
        yrs_to_death = float(20)
        print('Based on your input you will need your retirement income for ', yrs_to_death, 'years.')
    
    #contribution to retirement savings before retiring    
    additional_principle = float(input('How much will you be adding to your retirement savings per year before retiring? '))

    #inflation rate
    inflation = input('What is the inflation rate per year? [press enter if you want 3%.]')
    if inflation:
        inflation = float(inflation)
    else:
        inflation = float(3)
        print('Based on your input the inflation rate is', inflation, '%.')

    #tax rate
    tax_rate = input('Based on what you will need, what will be your tax rate? [press enter if you want to assume 25% for both state and federal taxes.]')
    if tax_rate:
        tax_rate = float(tax_rate)
    else:
        tax_rate = float(25)
        print('Based on your input your overall income tax rate is', tax_rate, '%.')
        
    final_principle = float(0)
    
    variables = [years_to_retire, how_much_needed_at_retirement, starting_principle,
                 how_much_ss_at_retirement, how_much_other_passive_income, investment_apr,
                 yrs_to_death, additional_principle, inflation, tax_rate, final_principle]
    
    variable_names = ['years_to_retire', 'how_much_needed_at_retirement', 'starting_principle',
                 'how_much_ss_at_retirement', 'how_much_other_passive_income', 'investment_apr',
                 'yrs_to_death', 'additional_principle', 'inflation', 'tax_rate', 
                 'final_principle']
    ''' 
    legend 
        variables[0] = years_to_retire 
        variables[1] = how_much_needed_at_retirement
        variables[2] = starting_principle
        variables[3] = how_much_ss_at_retirement
        variables[4] = how_much_other_passive_income
        variables[5] = investment_apr 
        variables[6] = yrs_to_death
        variables[7] = additional_principle
        variables[8] = inflation
        variables[9] = tax_rate
        variables[10] = final_principle
    '''
    df = pd.DataFrame(columns= ['years_to_retire', 'how_much_needed_at_retirement', 'starting_principle',
                 'how_much_ss_at_retirement', 'how_much_other_passive_income', 'investment_apr',
                 'yrs_to_death', 'additional_principle', 'inflation', 'tax_rate', 
                 'final_principle']) #creates the df using variable_names list
    
    return variables, variable_names, df

'''#initializes the dataframe
def init_df():
    
    variables, variable_names = var_input()
    df = pd.DataFrame(columns= [variable_names]) #creates the df using variable_names list
    
    return df, variables, variable_names'''

#determines the amount of gain over entirement retirement period (retirement to death)
def impact_years_compounding(variables, df):
    yr = 1
    starting_principle = variables[2]
    while yr <= variables[6]: #calculates compounding across multiple years
        if yr > variables[0] :#ends additional contribution to retirement at the start of retirement    
            additional_principle = 0
        else:
            additional_principle = variables[7]
          
        if yr < variables[0]: #years_to_retire: starts retirement debit at the start of retirement
            #variables[0] = when retirement starts
            debit = 0
            taxes = 0
            inflat = 0
        else:
            debit = (1 + (variables[9]/100) + (variables[8]/100)*((yr + 1) - variables[0])) * (-variables[1]) #how_much_needed_at_retirement
            # variables[9] are taxes; variables[1] is amount needed at retirement; variables[8] is inflation
            taxes = variables[9]
            inflat = variables[8]
            
        years_to_retire = variables[0] - yr
        if years_to_retire < 0:
            years_to_retire = 'Retired'
        
        years_left = variables[6] - yr
        
        final_principle, apr, soc_security, passive_retirement_income = annual_by_month_compound(variables, starting_principle, 
                                                        additional_principle, debit, yr)
        
        '''df = df.append({'APR': apr, 'Year Number': yr, 'Starting Balance': starting_principle, 
                            'Ending Balance': final_principle, 'Additional Premium' : additional_premium, 
                            'Debit' : debit} , ignore_index = True)'''
        
        df = df.append({'years_to_retire': years_to_retire, 
                        'how_much_needed_at_retirement' : debit, 
                        'starting_principle': variables[2],
                        'how_much_ss_at_retirement': soc_security, #variables[3] 
                        'how_much_other_passive_income': passive_retirement_income, #variables[4], 
                        'investment_apr': variables[5],
                        'yrs_to_death': years_left, 
                        'additional_principle': additional_principle, 
                        'inflation': inflat, 
                        'tax_rate': taxes, 
                        'final_principle': final_principle}, 
                        ignore_index = True)                           
        
        starting_principle = final_principle
        
        yr += 1
        
    return df
    
#defines gain through monthly compounding while taking into account additional principle
def annual_by_month_compound(variables, starting_principle, additional_principle, debit, yr): #interest rate compounded monthly for one year
    
    if yr > variables[0]:
        soc_security = variables[3]
        passive_retirement_income = variables[4]
    else: 
        soc_security = 0
        passive_retirement_income = 0
    mpr = variables[5]/1200 # APR divided by 12 to get monthly interest rate
    #print('mpr = ', mpr)
    month = 1    
    new_principle = starting_principle
    #print('new_principle = ', new_principle, '\n')
    while month < 13:
        gain_from_interest_on_principle = new_principle * mpr
        #print('gain_from_interest_on_principle =', gain_from_interest_on_principle)
        new_principle = new_principle + gain_from_interest_on_principle + additional_principle/12 + debit/12 + soc_security/12 + passive_retirement_income/12
        month += 1
        new_principle = float(new_principle)
    return new_principle, variables[5], soc_security, passive_retirement_income



In [107]:
#########################################
# MAIN #
#########################################

variables, variable_names, df = var_input()

df = impact_years_compounding(variables, df)

print('\n', df)

In how many years do you plan to retire? 40
How much per year will you need at retirement? [Before taxes.]100
How much have you saved for retirement right now? [in thousands]0
How much Social Security benefits per year will you get at retirement? [in thousands/year] 40
How much other passive income per year will you get at retirement? [e.g. Pension; in thousands/year] 0
What is the annualized percent return on your retirement saving before retiring? 15
How long will you need retirement income? [press enter if you want 20 yrs.]60
How much will you be adding to your retirement savings per year before retiring? 12
What is the inflation rate per year? [press enter if you want 3%.]
Based on your input the inflation rate is 3.0 %.
Based on what you will need, what will be your tax rate? [press enter if you want to assume 25% for both state and federal taxes.]
Based on your input your overall income tax rate is 25.0 %.

    years_to_retire  how_much_needed_at_retirement  starting_principle  \

59           0.0                   0.0        3.0      25.0    595509.788523  


In [106]:
display(df.head(40))

,years_to_retire,how_much_needed_at_retirement,starting_principle,how_much_ss_at_retirement,how_much_other_passive_income,investment_apr,yrs_to_death,additional_principle,inflation,tax_rate,final_principle
0,39.0,0.0,2.0,0.0,0.0,15.0,19.0,12.0,0.0,0.0,15.181870
1,38.0,0.0,2.0,0.0,0.0,15.0,18.0,12.0,0.0,0.0,30.482786
2,37.0,0.0,2.0,0.0,0.0,15.0,17.0,12.0,0.0,0.0,48.243393
3,36.0,0.0,2.0,0.0,0.0,15.0,16.0,12.0,0.0,0.0,68.859098
4,35.0,0.0,2.0,0.0,0.0,15.0,15.0,12.0,0.0,0.0,92.788870
5,34.0,0.0,2.0,0.0,0.0,15.0,14.0,12.0,0.0,0.0,120.565462
6,33.0,0.0,2.0,0.0,0.0,15.0,13.0,12.0,0.0,0.0,152.807266
7,32.0,0.0,2.0,0.0,0.0,15.0,12.0,12.0,0.0,0.0,190.232086
8,31.0,0.0,2.0,0.0,0.0,15.0,11.0,12.0,0.0,0.0,233.673115
9,30.0,0.0,2.0,0.0,0.0,15.0,10.0,12.0,0.0,0.0,284.097485


In [ ]:
df1 = df.drop('yrs_to_death')

In [ ]:
years_to_retire': variables[0], 
                        'how_much_needed_at_retirement' : debit, 
                        'starting_principle': variables[2],
                        'how_much_ss_at_retirement': variables[3], 
                        'how_much_other_passive_income': variables[4], 
                        'investment_apr': variables[5],
                        'yrs_to_death': variables[6], 
                        'additional_principle': additional_principle, 
                        'inflation': variables[8], 
                        'tax_rate': variables[9], 
                        'final_principle': final_principle},